# Pluri-Gaussian

<!-- SUMMARY: Pluri-Gaussian simulations performed in 2D -->

<!-- CATEGORY: Methodology -->

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gstlearn as gl
import gstlearn.plot as gp
import gstlearn.document as gdoc
import os

gdoc.setNoScroll()

## Prepare the basic gstlearn objects

Initial objects are located in a specific Directory which is defined in the next Container operation. Note that his operation will modify automatically all the names of the Files retreived using Serialize / Deserialize operation (not when reading using CSV). Also note that the Container name must be ended using a "/" (as its serves as a Directory).

In [ ]:
ndim = 2
gl.defineDefaultSpace(gl.ESpaceType.RN,ndim)

### Load the data file

This Load is performed starting from a CSV file.

In [ ]:
filename = gdoc.loadData("BRGM", "Nitrates_LANU.csv")
datCat = pd.read_csv(filename,sep=";")
datCat.head()

### Loading polygon from a file

The polygon is created by deserializing the Neutral Polygon File

In [ ]:
filename = gdoc.loadData("BRGM", "poly_LANU.ascii")
poly = gl.Polygons.createFromNF(filename)
poly

### Creation of the gstlearn data base 

In [ ]:
dat = gl.Db()
fields = ["X","Y","LANU"]
dat[fields] = datCat[fields].values

###  Specification of the role of each variable (named "locators" in gstlearn)

In [ ]:
dat.setLocators(["X","Y"],gl.ELoc.X) #Coordinates
dat.setLocator("LANU",gl.ELoc.Z) #Variable of interest
dat

### Creation of the output grid

The output grid will contain 47 x 101 nodes. It is built to cover the data file plus an extension of 10000 x 10000.

In [ ]:
Result = gl.DbGrid.createCoveringDb(dat,[47,101],[],[],[50000,50000])

### Add a selection (mask the cells outside the polygon)

The initial selection (based on the application of the Polygon to the grid data base) must be dilated in order to avoid edge effect.

In [ ]:
gl.db_polygon(Result,poly)
Result.setLocator("Polygon",gl.ELoc.Z)
Result.morpho(gl.EMorpho.DILATION,0.5,1.5,option=0,verbose=False,radius=[1,1])
Result["Polygon"] = Result["Morpho.Polygon.*"]
Result.deleteColumn("Morpho.Polygon.*")
Result.setLocator("Polygon",gl.ELoc.SEL)

In [ ]:
ax = Result.plot(nameRaster="Polygon",useSel=False,flagLegendRaster=False)
ax.gstpoint(dat,nameColor="LANU",size=2)
ax.polygon(poly,linewidth=1,edgecolor="r")
ax.geometry(dims=[10,10])
ax.decoration(title="Initial information")

## Computation of the proportions

### Compute global proportions (for information)

In [ ]:
propGlob = gl.dbStatisticsFacies(dat)
ncat = len(propGlob)
for i in range(ncat):
    print("Proportion of facies "+str(i+1),"=",propGlob[i])

### Compute local proportions

The next parts will be simplified in a future dedicated API

**2.2.1 Creation of the spatial regularization model for proportions**

In [ ]:
model = gl.Model.createFromDb(Result)
cova = gl.CovAniso.createIsotropic(model.getContext(),gl.ECov.MATERN,range=50000.,param=2.,sill=1.,) 
model.addCov(cova)

In [ ]:
err = gl.db_proportion_estimate(dat,Result,model)

In [ ]:
dbfmt = gl.DbStringFormat()
dbfmt.setFlags(flag_stats=True)
dbfmt.setNames(["Prop.*"])
Result.display(dbfmt)

### Display the results

In [ ]:
for i in range(ncat):
    fig, ax = plt.subplots()
    ax = Result.plot(nameRaster="Prop."+str(i+1))
    ax.decoration(title="Proportion Facies #"+str(i+1))
    ax.geometry(dims=[10,10], aspect=1)
    ax.gstpoint(dat,nameColor="LANU",size=2,color="black")
    dat.addSelectionByLimit("LANU",gl.Limits((i+1,i+1)),"SelPoint")
    ax.gstpoint(dat,nameColor="LANU",size=0.8,color="red")
    dat.deleteColumn("SelPoint")
    ax.polygon(poly,linewidth=1,edgecolor="r")

Creating the environment to infer the Rule. It uses a variogram calculated over very few lags close to the origin.

In [ ]:
varioParam = gl.VarioParam()
dirparam = gl.DirParam.create(nlag = 2, dpas=100)
varioParam.addDir(dirparam);
ruleprop = gl.RuleProp.createFromDb(Result);
ruleprop.fit(dat, varioParam, 1);
ngrf = ruleprop.getRule().getNGRF()
print("Number of GRF =",ngrf)

In [ ]:
ax=ruleprop.getRule().plot()

In [ ]:
dirparam = gl.DirParam.create(nlag = 19, dpas=1000)
covparam = gl.VarioParam();
covparam.addDir(dirparam);
cov = gl.variogram_pgs(dat,covparam,ruleprop);

In [ ]:
cov.display()

We extract the experimental variograms of each GRF.

In [ ]:
vario1 = gl.Vario.createReduce(cov,[0],[],True)
if ngrf > 1:
    vario2 = gl.Vario(cov)
    vario2.resetReduce([1],[],True)

In [ ]:
vario1.display()
if ngrf > 1:
    vario2.display()

We now fit the model of each GRF considered as independent. 

The fit is performed under the constraint that the sill should be 1.

In [ ]:
ctxt = gl.CovContext(1,2) # use default space
constraints = gl.Constraints()
constraints.setConstantSillValue(1.)
covs  = [gl.ECov.MATERN]

modelPGS1 = gl.Model(ctxt)
modelPGS1.fit(vario1,covs,constraints)
modelPGS1.display()

if ngrf > 1:
    modelPGS2 = gl.Model(ctxt)
    modelPGS2.fit(vario2,covs,constraints)
    modelPGS2.display()
else:
    modelPGS2 = None

For each GRF, we can plot the experimental variogram as well as the fitted model.

In [ ]:
ax = gp.varmod(vario1,modelPGS1)
if ngrf > 1:
    ax = gp.varmod(vario2,modelPGS2)

In this paragraph, we compare the experimental indicator variogram to the one derived from the Model of the underlying GRFs.

In [ ]:
dirparamindic = gl.DirParam.create(nlag=19, dpas=1000)
varioparamindic = gl.VarioParam()
varioparamindic.addDir(dirparamindic)
varioindic = gl.Vario(varioparamindic)
err = varioindic.computeIndic(dat)

In [ ]:
varioindic2 = gl.model_pgs(dat, varioparamindic, ruleprop, modelPGS1, modelPGS2);

In [ ]:
axs = gp.varmod(varioindic,varioLinestyle='solid')
gp.geometry(axs,dims=[10,10])
axs = gp.varmod(varioindic2,varioLinestyle='dashed',axsOld=axs)

In [ ]:
neigh = gl.NeighUnique.create()
err = gl.simpgs(dat,Result,ruleprop,modelPGS1,modelPGS2,neigh)

In [ ]:
ax = Result.plot()
ax.geometry(dims=[10,10])